In [1]:
import tensorflow as tf
from tensorflow import keras
import string
import numpy as np
import pandas as pd
import re
import nltk
import pickle
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

In [2]:
data = pd.read_csv("labeled_data.csv")
data.drop(labels=["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"], axis=1, inplace=True)
data["class"].replace({1: 0}, inplace=True)
data["class"].replace({2: 1}, inplace=True)
print(data.head)


<bound method NDFrame.head of        class                                              tweet
0          1  !!! RT @mayasolovely: As a woman you shouldn't...
1          0  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2          0  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3          0  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4          0  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...      ...                                                ...
24778      0  you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779      1  you've gone and broke the wrong heart baby, an...
24780      0  young buck wanna eat!!.. dat nigguh like I ain...
24781      0              youu got wild bitches tellin you lies
24782      1  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...

[24783 rows x 2 columns]>


In [3]:
'''
Code in this cell taken from
jatingoyal123. “Hate&Offensive_language.” Kaggle.com, Kaggle, 26 May 2021, www.kaggle.com/jatingoyal123/hate-offensive-language. Accessed 24 June 2021.
'''
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

    
data['tweet']=data['tweet'].apply(clean_text)

In [4]:
xTrain, xTest, yTrain, yTest = train_test_split(data["tweet"], data["class"], test_size=.2, random_state=42)
v = CountVectorizer(stop_words="english", ngram_range=(1, 3))
#tweak ngram^^
v_xTrain = v.fit_transform(xTrain).toarray()
v_xTest = v.fit_transform(xTest)

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
t = Tokenizer(num_words=50000)
t.fit_on_texts(xTrain)
sequences_matrix = sequence.pad_sequences(t.texts_to_sequences(xTrain),maxlen=300)
model = Sequential()
model.add(Embedding(50000, 100, input_length=300))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [15]:
stop = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5
)

checkpoint= ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history=model.fit(x=sequences_matrix, y=yTrain, batch_size=1024,epochs=10,
          validation_split=0.2, callbacks=[stop,checkpoint])

!mkdir -p saved_model
model.save('saved_model/speech_model')
model.save_weights

Epoch 1/10
16/16 [==============================] - 936s 61s/step - loss: 0.5679 - accuracy: 0.7799 - val_loss: 0.4401 - val_accuracy: 0.8361
Epoch 2/10
16/16 [==============================] - 870s 54s/step - loss: 0.4430 - accuracy: 0.8358 - val_loss: 0.4343 - val_accuracy: 0.8361
Epoch 3/10
16/16 [==============================] - 1073s 66s/step - loss: 0.4363 - accuracy: 0.8287 - val_loss: 0.4060 - val_accuracy: 0.8361
Epoch 4/10
16/16 [==============================] - 1191s 74s/step - loss: 0.3948 - accuracy: 0.8308 - val_loss: 0.3129 - val_accuracy: 0.8417
Epoch 5/10
16/16 [==============================] - 1037s 62s/step - loss: 0.3024 - accuracy: 0.8542 - val_loss: 0.2181 - val_accuracy: 0.8886
Epoch 6/10
16/16 [==============================] - 854s 53s/step - loss: 0.2154 - accuracy: 0.9094 - val_loss: 0.1786 - val_accuracy: 0.9281
Epoch 7/10
16/16 [==============================] - 900s 55s/step - loss: 0.1675 - accuracy: 0.9362 - val_loss: 0.1529 - val_accuracy: 0.9425
Epo

TypeError: save_weights() missing 1 required positional argument: 'filepath'

In [17]:
test_sequences = t.texts_to_sequences(xTest)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=300)
accr = model.evaluate(test_sequences_matrix,yTest)

155/155 [==============================] - 39s 218ms/step - loss: 0.1481 - accuracy: 0.9399


In [23]:
new_model = tf.keras.models.load_model('saved_model/speech_model')
test_sequences = t.texts_to_sequences(xTest)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=300)
accr = new_model.evaluate(test_sequences_matrix, yTest)

155/155 [==============================] - 29s 189ms/step - loss: 0.1481 - accuracy: 0.9399


In [14]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [13]:
def clean_text2(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

new_model = tf.keras.models.load_model('saved_model/speech_model')

with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)
# As a woman you shouldn't complain about cleaning up your house
test=[clean_text2("Monkeys are adorable!")]
print(test)
seq = load_tokenizer.texts_to_sequences(test)
padded = sequence.pad_sequences(seq, maxlen=300)
print(seq)
pred = new_model.predict(padded)
print("pred", pred)
if pred<0.3:
    print("hate or hate speech")
else:
    print("all clear")


Monkeys are adorable!
monkeys are adorable
['monkey ador']
[[170, 1906]]
pred [[0.7242426]]
all clear
